# Tutorial for Project 2

## Outline
1. Prerequisites
2. Clone the GitHub Repository and Install Dependencies
3. User Agent
4. Tool-calling DB Agent using GPT-4o-mini
5. Running Inference
6. Viewing the Conversation Trajectories
7. Submission

## Prerequisites

### Set OpenAI API key

In [4]:
import getpass
from IPython.display import clear_output

clear_output()
# Please enter your API key
new_api_key = ''
while len(new_api_key) == 0:
    new_api_key = getpass.getpass("Please input your API key: ")
    clear_output()

### Clone the GitHub Repository and Install Dependencies

In [ ]:
%cd /content
!rm -rf ai612_project_2

In [ ]:
# Cloning the GitHub repository
!git clone -q https://github.com/benchay1999/ai612_project_2.git
%cd ai612_project_2

# Installing dependencies
! pip install -q -r requirements.txt

In [ ]:
import os
if os.path.exists('.env'):
    os.remove('.env')
with open('.env', 'w') as f:
    f.write(f'OPENAI_API_KEY="{new_api_key}"')

## User LLM
This User LLM simulates the behavior of users who interact with text-to-SQL systems without SQL knowledge. Let's take a look at the system prompt of the User LLM.

In [1]:
from src.envs.user import LLMUser
# dummy user agent
user_agent = LLMUser(model="gpt-4o-mini")
# system prompt for the user agent
print(f"system prompt:\n{user_agent.build_system_prompt('Your goal is to find the gender of a patient. Specifically, you want to know the gender of the patient with ID 10027602.')}")

system prompt:
You are a human user who wants to retrieve data from an EHR database by interacting with an SQL agent.
User instruction: 

Instruction: Your goal is to find the gender of a patient. Specifically, you want to know the gender of the patient with ID 10027602.

Rules:
- Current time is 2100-12-31 23:59:00.
- You don't know SQL at all and only have a rough idea of what information the database contains.
- Explain your intent in plain language so that the SQL agent understands exactly what you need.
- Generate one request at a time to simulate the user's message.
- Do not be too wordy in your messages. Be concise and to the point.
- Do not give away all the instruction at once. Only provide the information necessary to ask or respond to the SQL agent.
- Do not hallucinate information that is not provided in the instructions.
- Do not repeat the exact instruction in the conversation. Instead, use your own words to convey the same information.
- Even if the SQL agent transfers t

The "user prompt" for the User LLM is "Hi! How can I help you today?"
This way, the User LLM generates a natural language question to address the "User instruction" in the above system prompt. 

Note that you can't modify the system prompt and the user prompt of the User LLM. Also, the User LLM should only be ran by GPT-4o-mini.

## Dataset
The sample validation dataset is in the `src/envs/mimic_iv/valid_data.json`. It contains 10 samples for this task. The `src/envs/mimic_iv/test_data.json` contains the same information as in the validation dataset, but with gold SQL and gold answer field removed.

IMPORTANT: When created your own custom validation set, make sure that the name of the JSON file is `src/envs/mimic_iv/valid_data.json`. Plus, when given test data, it should be exactly stored in `src/envs/mimic_iv/test_data.json`.

In [11]:
import json
with open("src/envs/mimic_iv/valid_data.json", 'r') as f:
    valid_data = json.load(f)
with open("src/envs/mimic_iv/test_data.json", 'r') as f:
    test_data = json.load(f)

print(valid_data[:2])
print(test_data[:2])

[{'task_id': '0', 'instruction': 'Your goal is to find the gender of a patient. Specifically, you want to know the gender of the patient with ID 10027602.', 'gold_sql': 'SELECT gender FROM patients WHERE subject_id = 10027602', 'gold_answer': [['f']]}, {'task_id': '1', 'instruction': 'Your goal is to find all the routes of administration for isosorbide dinitrate for patients in the database.', 'gold_sql': "SELECT DISTINCT prescriptions.route FROM prescriptions WHERE prescriptions.drug = 'isosorbide dinitrate'", 'gold_answer': [['po/ng']]}]
[{'task_id': '0', 'instruction': 'Your goal is to find the gender of a patient. Specifically, you want to know the gender of the patient with ID 10027602.'}, {'task_id': '1', 'instruction': 'Your goal is to find all the routes of administration for isosorbide dinitrate for patients in the database.'}]


Below is the formatted version of each sample.

In [15]:
from src.types import Task
valid_tasks = [Task(**task) for task in valid_data]
print(valid_tasks[:2])
test_tasks = [Task(**task) for task in test_data]
print(test_tasks[:2])


[Task(task_id='0', instruction='Your goal is to find the gender of a patient. Specifically, you want to know the gender of the patient with ID 10027602.', gold_sql='SELECT gender FROM patients WHERE subject_id = 10027602', gold_answer=[['f']]), Task(task_id='1', instruction='Your goal is to find all the routes of administration for isosorbide dinitrate for patients in the database.', gold_sql="SELECT DISTINCT prescriptions.route FROM prescriptions WHERE prescriptions.drug = 'isosorbide dinitrate'", gold_answer=[['po/ng']])]
[Task(task_id='0', instruction='Your goal is to find the gender of a patient. Specifically, you want to know the gender of the patient with ID 10027602.', gold_sql=None, gold_answer=None), Task(task_id='1', instruction='Your goal is to find all the routes of administration for isosorbide dinitrate for patients in the database.', gold_sql=None, gold_answer=None)]


## Baseline DB Agent - Tool-calling
Now, we will take a look into the tool-calling agent. It uses the following 4 tools:
- `sql_db_list_tables`: Get the list of table names in the database.
- `sql_db_schema`: Get the columns of a specific table and its sample rows.
- `value_substring_search`: Retrieve up to k values from a specific column that contains the specified substring.
- `sql_db_query`: Execute a SQL query against the database and get back the result. If the query is not correct, an error message will be returned. The maximum number of results to return is 100.

First, let's take a look at the system prompt of the baseline agent. It contains "domain policies" i.e., rules that the DB agent should follow in the MIMIC_IV database. You can see the rules in the "Rules" section of the system prompt. We don't recommend changing the domain policies, but you can freely change the system prompt above the "Rules" section.

In [2]:
from src.agents.tool_calling_agent import ToolCallingAgent
with open("src/envs/mimic_iv/rules.txt", 'r') as f:
    db_agent_rules = f.read()
# dummy tool-calling agent
tc_agent = ToolCallingAgent(tools_info=[], rule=db_agent_rules, model="gpt-4o-mini")
# system prompt for the tool-calling agent
print(f"system prompt:\n{tc_agent.instruction}")

system prompt:
- You are a SQL agent that translates natural language questions into precise SQL queries for electronic health records (EHR).
- You are currently engaged in a conversation with a user who wants to retrieve data from an EHR database.
- If the user's request is ambiguous or missing crucial information (e.g., filtering criteria), you must ask clarifying questions in plain language.
- You can interact with the database to learn more about its schema or the values stored in it by using the tools provided.
- Do not invent or fabricate any information not provided by the user or the tools.
- You should make at most one tool call at a time.
- If you do call a tool, do not respond to the user in that same turn.
- Do not generate SQL queries directly without knowing the database schema and values intended to be used in the SQL query by calling substring_search_tool.
- When the user asks for specific diagnoses, procedures, medications, or lab tests, try your best to use the tool t

Below are the tools the DB Agent uses. It can be found in the `src/envs/mimic_iv/tools/` folder. When implementing your own tools, we recommend to create the necessary files in the same folder as this.

In [10]:
from src.envs.mimic_iv.tools.sql_db_list_tables import SqlDbListTables
from src.envs.mimic_iv.tools.sql_db_schema import SqlDbSchema
from src.envs.mimic_iv.tools.value_substring_search import ValueSubstringSearch
from src.envs.mimic_iv.tools.sql_db_query import SqlDbQuery

print(f"sql_db_list_tables:\n{SqlDbListTables.get_info()}")
print(f"sql_db_schema:\n{SqlDbSchema.get_info()}")
print(f"value_substring_search:\n{ValueSubstringSearch.get_info()}")
print(f"sql_db_query:\n{SqlDbQuery.get_info()}")

sql_db_list_tables:
{'type': 'function', 'function': {'name': 'sql_db_list_tables', 'description': 'Get the list of table names in the database.', 'parameters': {'type': 'object', 'properties': {'tool_input': {'type': 'string', 'description': 'An empty string; no input required.'}}, 'required': []}}}
sql_db_schema:
{'type': 'function', 'function': {'name': 'sql_db_schema', 'description': 'Get the columns of a specific table and its sample rows.', 'parameters': {'type': 'object', 'properties': {'table_names': {'type': 'string', 'description': 'A comma-separated list of table names to retrieve schema and sample rows for.'}}, 'required': ['table_names']}}}
value_substring_search:
{'type': 'function', 'function': {'name': 'substring_search_tool', 'description': 'Retrieve up to k values from a column that contains the specified substring.', 'parameters': {'type': 'object', 'properties': {'table': {'type': 'string', 'description': 'The table name.'}, 'column': {'type': 'string', 'description

By feeding this to the `tools_info` in the DB Agent, the agent decides when or when not to use the tools.

## Inference & Evaluation

Now, let's see how the User LLM and the DB agent interacts.

`run.py` simulates the conversation between the User LLM and the DB Agent. However, you should feed necessary information when running your own DB agent:
- model: The backbone model of the DB Agent
- agent_strategy: the name of your DB agent. See `src/agent_factory.py` for details. When implemented your own DB agent, you should fill in the TODO: field.
- max_concurrency: the number of concurrent samples to run at once. Larger number leads to faster inference.
- eval_mode: "valid" or "test". When running inference on valid_data.json, where you have the gold SQL and the gold answer, it should be "valid". On the other hand, when running inference on test_data.json, where you don't have the gold SQL and the gold answer, it should be "test". When running on the test set, no evaluation (i.e., outputting final scores) is done. However, you can still see the inference results.


Check `run_mimic_iv.sh` for more details.

In [ ]:
# running the shell script `run_mimic_iv.sh`. This might take a while.
! bash run_mimic_iv.sh

When the inference is done, the results (conversation trajectories between the User LLM and the DB Agent) are saved in the `results/` folder. The name of the result file contains meta information about the inference. You should change the name of the file to `team_{team_number}.json` e.g., `team_1.json` and upload it to KLMS.

In [6]:
# checking the total cost of the inference
import json
import pandas as pd
path = 'results/mimic_iv-tool-calling-gpt-4o-mini-0.0_range_0--1_user-gpt-4o-mini-llm_0421105745_valid.json'
with open(path, 'r') as f:
    results = json.load(f)
print(f'Total cost: ${round(sum([l["cost"]["total_cost"] for l in results if pd.notna(l["cost"]["total_cost"])]), 2)}')

Total cost: $0.19


## What you need to do:
1. Implement your own agent in `src/agents/` (`TODO_implement_agent.py`)
2. If needed, implement the tools your agent will use in `src/envs/mimic_iv/tools/` (`TODO_implement_tool.py`)
3. Change `src/envs/mimic_iv/env.py` to update the tools the DB agent can use in the environment.
4. Change `src/agent_factory.py` to make it able to call your implemented agent.
5. Change `run_mimic_iv.sh` to do inference & evaluation.

In [1]:
# viewing conversation trajectories
! streamlit run visualizer.py --server.port 8505




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8505
  Network URL: http://192.168.0.10:8505
  External URL: http://59.29.246.30:8505

^C
  Stopping...
